WEBSCRAPING MERCADOLIBRE

In [239]:
import pandas as pd
import matplotlib as plt
from bs4 import BeautifulSoup
import requests

Create the headers

In [240]:
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

Get links

In [241]:
productlinks = []
pages = ('', 'Desde_51_', 'Desde_101_', 'Desde_151_')
for x in pages:
    r = requests.get(f'https://listado.mercadolibre.com.co/consolas-videojuegos/consolas/sony/pplaystation-5_{x}NoIndex_True')
    soup = BeautifulSoup(r.content)
    productlist = soup.find_all('div', class_='ui-search-result__content-wrapper')
    for item in productlist:
        for link in item.find_all('a', href=True):
            productlinks.append(link['href'])

Get information by links

In [242]:
productlist = []
for link in productlinks:
    url = link
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content)
    generalinfo = soup.find('div', class_='ui-pdp-header__info')
    try:
        price = soup.find('span', class_='andes-money-amount__fraction').text.strip()
    except:
        price = 0
    # color = soup.find('span', class_='picker-label-COLOR_SECONDARY_COLOR').text.strip()
    try:
        name = soup.find('h1').text.strip()
    except:
        name = soup.find('div', class_='ui-pdp-header__title-container').text.strip()
    try:
        reviews = generalinfo.find('span', class_='ui-pdp-review__amount').text.strip()
    except:
        reviews = 0
    try:
        rating = generalinfo.find('span', class_='ui-pdp-review__rating').text.strip()
    except:
        rating = 0
    try:
        state = soup.find('span', class_='ui-pdp-subtitle').text.strip()
    except:
        state = 'No Defined'
    try:
        discount = soup.find('span', class_='andes-money-amount__discount').text.strip()
    except:
        discount = 0
    tables = soup.find_all('table', class_='andes-table')
    product = {
        'name' : name,
        'price' : price,
        'rating' : rating,
        'reviews' : reviews,
        'state' : state,
        'discount' : discount,
        #'color' : color
    }
    rowstable = soup.find_all('tr', class_='andes-table__row ui-vpp-striped-specs__row')
    for row in rowstable:
        key = row.find('th').text
        value = row.find('td').text
        product[key] = value
    print(product, link)
    productlist.append(product)
print("Successfull Scrapper")

{'name': 'Sony PlayStation 5 Slim 1TB Call of Duty: Modern Warfare III Bundle color  blanco', 'price': '3.299.900', 'rating': 0, 'reviews': 0, 'state': 'Nuevo  |  +25 vendidos', 'discount': '21% OFF', 'Marca': 'Sony', 'Línea': 'PlayStation', 'Modelo': 'PlayStation 5', 'Submodelo': 'Slim', 'Edición': 'Call of Duty: Modern Warfare III Bundle', 'Color': 'Blanco', 'Conectividad de red móvil': '4G', 'Con Wi-Fi': 'Sí', 'Con HDMI': 'Sí', 'Con USB': 'Sí', 'Con Bluetooth': 'Sí', 'Versión de Bluetooth': '4', 'Incluye controles': 'Sí', 'Cantidad de controles incluidos': '1', 'Conectividades de los controles': 'Inalámbrico', 'Procesador': 'AMD Ryzen Zen 2', 'CPU': 'AMD Ryzen Zen 2', 'GPU': 'AMD Radeon', 'Velocidad del procesador': '3.5 GHz', 'Cantidad de núcleos de CPU': '8', 'Capacidad': '1 TB', 'Tipos de memoria': 'GDDR6', 'Memoria RAM': '16 GB', 'Accesorios incluidos': '1 joystick', 'Tipo de consola': 'De sobremesa', 'Año de lanzamiento': '2023', 'Títulos de juegos incluidos': 'Call of Duty: Mo

Convert the data into a df

In [243]:
df = pd.DataFrame(productlist)
df.head()

,name,price,rating,reviews,state,discount,Marca,Línea,Modelo,Submodelo,...,Con S/PDIF,Tipo de batería,Duración máxima de la batería,Modelos alfanuméricos,Aplicaciones soportadas,Capacidad de la batería,Tamaños de las pantallas,Tipos de tarjetas de memoria,Tipo de pila,Cantidad de pantallas
0,Sony PlayStation 5 Slim 1TB Call of Duty: Mode...,3.299.900,0,0,Nuevo | +25 vendidos,21% OFF,Sony,PlayStation,PlayStation 5,Slim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Consola Sony Playstation 5 Slim Version Lector...,2.989.900,0,0,Nuevo | +100 vendidos,24% OFF,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Consola Playstation 5 Slim Disco Spiderman - P...,3.199.900,4.9,(70),Nuevo | +100 vendidos,25% OFF,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sony PlayStation 5 825GB Digital Edition color...,2.200.000,4.9,(237),Nuevo | +1000 vendidos,24% OFF,Sony,PlayStation,PlayStation 5,NaN,...,No,Ion de litio,6 h,CFI-2014,NaN,NaN,NaN,NaN,NaN,NaN
4,Sony PlayStation 4 1TB,1.680.000,0,0,Nuevo | +1000 vendidos,24% OFF,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [245]:
df.describe()

,name,price,rating,reviews,state,discount,Marca,Línea,Modelo,Submodelo,...,Con S/PDIF,Tipo de batería,Duración máxima de la batería,Modelos alfanuméricos,Aplicaciones soportadas,Capacidad de la batería,Tamaños de las pantallas,Tipos de tarjetas de memoria,Tipo de pila,Cantidad de pantallas
count,110,110,110,110,110,110,20,15,20,16,...,14,7,1,2,13,2,1,2,2,2
unique,102,81,4,9,13,16,1,2,6,7,...,1,2,1,2,6,2,1,1,1,1
top,Smart Buy Tienda Oficial,3.299.900,0,0,Nuevo,0,Sony,PlayStation,PlayStation 5,PS5 SLIM DISCO,...,No,Polímero de litio,6 h,CFI-2014,"Netflix,YouTube",4310 mAh,0.203 m,SD,AA,0
freq,3,3,101,101,43,79,20,14,10,6,...,14,4,1,1,4,1,1,2,2,2
